# Data Extractions from OpenEO

To run the extractions, you need an account in the [Copernicus Data Space Ecosystem (CDSE)](https://dataspace.copernicus.eu/).

In [ ]:
!pip install seaborn --quiet
!pip install git+https://github.com/WorldCereal/prometheo.git@scaleag_augmentations --quiet
!pip install git+https://github.com/ScaleAGData/scaleag-vito.git@prometheo-integration --quiet

In [ ]:
from loguru import logger
import geopandas as gpd
from pathlib import Path
from scaleagdata_vito.openeo.extract_sample_scaleag import (
    generate_input_for_extractions,
    extract
)
from scaleagdata_vito.presto.presto_df import load_dataset
# from scaleagdata_vito.presto.utils import evaluate_finetuned_model
# from prometheo.datasets.scaleag import ScaleAgDataset
# from prometheo import finetune
# from prometheo.finetune import Hyperparams
# from prometheo.models.presto import param_groups_lrd
# from prometheo.models.presto.wrapper import PretrainedPrestoWrapper, load_pretrained
# from torch import nn
# from torch.optim import AdamW, lr_scheduler

### Assess data correctness before launching the OpenEO jobs 
You can run some checks on your input file to make sure they are suitable to run the extractions successfully. In particular, it is important to check the validity of the geometries and, ideally, to also have a column containing a unique id for each sample.

In case of invalid geometries, you will be provided with both the dataframe with the failing polygons to be fixed and the one with valid geometries.


In [2]:
def check_unique_id(df_path, unique_id):
    df = gpd.read_file(df_path)
    if df[unique_id].nunique() != df.shape[0]:
        logger.info("IDs are not unique!")
        return df[df[unique_id].duplicated(keep=False)]
    else:
        logger.info("IDs are unique")
        return None

def check_valid_geometry(df):
    if isinstance(df, str):
        df = gpd.read_file(df)
    df_invalid = df[~df.geometry.is_valid]
    # Assessing wheather some invalid geometries are present
    if len(df_invalid) > 0:
        # 1) some invalid geometries are present. Attempt fixing them
        df['geometry'] = df.geometry.buffer(0)
        df_invalid = df[~df.geometry.is_valid]
        df_valid = df[df.geometry.is_valid]
        if len(df_invalid) > 0:
            # 2) Still some invalid geometries are present. Return them
            logger.info("Invalid geometries found! Returning invalid geometries")
            return df_invalid, df_valid
        else:
            # All geometries are now valid. Return fixed dataframe and empty dataframe for invalid geometries
            logger.info("Fixed some invalid geometries. All geometries are now valid")
            return gpd.GeoDataFrame(), df
    else:
        # All geometries are valid. Return empty dataframe for invalid geometries
        logger.info("All geometries are valid")
        return gpd.GeoDataFrame(), df

def _save(save_to, original_file_path, df, suffix=''):
    if suffix!='':
        filename = Path(save_to) / f"{Path(original_file_path).stem}_{suffix}.geojson"
    else:
        filename = Path(save_to) / f"{Path(original_file_path).stem}.geojson"
    logger.info(f"Saving invalid geometries to {filename}")
    Path(save_to).mkdir(parents=True, exist_ok=True)
    df.to_file(filename)

In [3]:
input_file = "/home/giorgia/Private/data/geomaize/correct/Maize_North_Ghana_valid.geojson"
invalid_geom, valid_geom = check_valid_geometry(input_file)
non_unique_ids = check_unique_id(input_file, unique_id="Field_id")

2025-03-20 09:49:41.818 | INFO     | __main__:check_valid_geometry:30 - All geometries are valid
2025-03-20 09:49:41.850 | INFO     | __main__:check_unique_id:7 - IDs are unique


In [46]:
# save files after geometry validity check. If invalid geometries are present, save them to a separate file
if len(invalid_geom) > 0:
    _save(
        save_to="/home/giorgia/Private/data/geomaize/invalid/",
        original_file_path=input_file,
        df=invalid_geom,
        suffix='',
    )

# save valid geometries to a separate file
_save(
    save_to="/home/giorgia/Private/data/geomaize/correct/",
    original_file_path=input_file,
    df=valid_geom,
    suffix='',
)

2025-03-17 10:19:18 | INFO     | __main__ - Saving invalid geometries to /home/giorgia/Private/data/geomaize/correct/Maize_North_Ghana_valid.geojson


#### Provide job instructions and start OpenEO extractions

1) Indicate the following fields in order to guide the extraction
2) In the cell below you will be asked for authentication and be provided with a link. click on the link and login with your CDSE credentials.  
3) Once the extraction process will be over, you will find your extracted dataset in the output folder you indicated. You can load it by running `load_dataset` as shown below

    ```python
    job_params = dict(
        output_folder=..., # where to save the extracted dataset
        input_df=..., # input georeferenced dataset to run the extractions for 
        start_date=..., # string indicating from which date to extract data  
        end_date=..., # string indicating until which date to extract the data 
        unique_id_column=..., # name of the column in the input_df containing the unique ID of the samples  
        composite_window=..., # "month" or "dekad" are supported. Default is "dekad"
    )
    ```

In [ ]:
# Dataset Parameters
task_type = "regression"
start_date="2021-07-01"
end_date="2021-10-31"
composite_window="dekad"
unique_id_column="Field_id"
input_df="/home/giorgia/Private/data/geomaize/correct/Maize_North_Ghana_valid.geojson"
output_folder="/home/giorgia/Private/data/geomaize/extractions_North_Ghana/"

job_params = dict(
    output_folder=output_folder,
    input_df=input_df,
    start_date=start_date,
    end_date=end_date,
    unique_id_column=unique_id_column,
    composite_window=composite_window,
)
extract(generate_input_for_extractions(job_params))

In [6]:
train_path = "/home/giorgia/Private/data/geomaize/extractions_2021"
valid_path = "/home/giorgia/Private/data/geomaize/extractions_2022"
test_path = "/home/giorgia/Private/data/geomaize/extractions_2023"

train_df = load_dataset(
    train_path,
    composite_window=composite_window
    )

val_df = load_dataset(
    valid_path,
    composite_window=composite_window
    )

test_df = load_dataset(
    test_path,
    composite_window=composite_window
    )

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 14.94it/s]


In [7]:

train_ds = ScaleAgDataset(
    dataframe=train_df,
    num_timesteps=train_df.available_timesteps.max(),
    task_type=task_type,
    target_name="Yield kg/H",
    compositing_window=composite_window,
)

val_ds = ScaleAgDataset(
    dataframe=val_df,
    num_timesteps=train_df.available_timesteps.max(),
    task_type=task_type,
    target_name="Kg/ha",
    compositing_window=composite_window,
)

test_ds = ScaleAgDataset(
    dataframe=test_df,
    num_timesteps=train_df.available_timesteps.max(),
    task_type=task_type,
    target_name="Kg/ha",
    compositing_window=composite_window,
)


2025-03-17 14:18:17.396 | INFO     | prometheo.datasets.scaleag:set_num_outputs:132 - Setting number of outputs to 1 for regression task.


2025-03-17 14:18:17.402 | INFO     | prometheo.datasets.scaleag:set_num_outputs:132 - Setting number of outputs to 1 for regression task.
2025-03-17 14:18:17.406 | INFO     | prometheo.datasets.scaleag:set_num_outputs:132 - Setting number of outputs to 1 for regression task.


In [8]:
# Fine Tuning Hyperparameters for
lr = 1e-4
batch_size = 32
epochs = 50
num_workers = 2
patience = 10
pretrained_model_path = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/scaleagdata/models/presto-ss-wc_10D.pt"
output_dir = Path("/home/giorgia/Private/data/geomaize/models/")
experiment_name = "presto-ss-wc-10D-ft-dek_geomaize"

In [9]:
# Construct the model with finetuning head
model = PretrainedPrestoWrapper(
    num_outputs=train_ds.num_outputs,
    regression=True if task_type == "regression" else False,
)
model = load_pretrained(model, pretrained_model_path, strict=False)

# Reduce epochs for testing purposes
hyperparams = Hyperparams(max_epochs=epochs, batch_size=batch_size, patience=patience, num_workers=num_workers, lr=lr)


# set loss depending on the task type
if task_type == "regression":
    loss_fn = nn.MSELoss()
elif task_type == "binary":
    loss_fn = nn.BCEWithLogitsLoss()
else:
    loss_fn = nn.CrossEntropyLoss()

parameters = param_groups_lrd(model)
optimizer = AdamW(parameters, lr=hyperparams.lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

finetuned_model = finetune.run_finetuning(
            model,
            train_ds,
            val_ds,
            experiment_name=experiment_name,
            output_dir=output_dir,
            loss_fn=loss_fn,
            hyperparams=hyperparams,
        )

evaluate_finetuned_model(
    finetuned_model=finetuned_model,
    test_ds=test_ds,
    num_workers=num_workers,
    batch_size=batch_size,
)

2025-03-17 14:18:45 | INFO     | prometheo.utils - Logging setup complete. Logging to: /home/giorgia/Private/data/geomaize/models/logs/presto-ss-wc-10D-ft-dek_geomaize.log and console.
2025-03-17 14:18:45 | INFO     | prometheo.finetune - Using output dir: /data/users/Private/giorgia/data/geomaize/models


Train metric: 0.066, Val metric: 0.179, Best Val Loss: 0.024 (no improvement for 9 epochs):  30%|███       | 15/50 [00:10<00:22,  1.54it/s]

2025-03-17 14:18:56 | INFO     | prometheo.finetune - Early stopping!


Train metric: 0.066, Val metric: 0.179, Best Val Loss: 0.024 (no improvement for 9 epochs):  30%|███       | 15/50 [00:11<00:26,  1.33it/s]


2025-03-17 14:18:56 | INFO     | prometheo.finetune - Finetuning done
2025-03-17 14:18:56 | INFO     | scaleagdata_vito.presto.utils - Evaluating the finetuned model on regression task


{'RMSE': 2161.155029296875,
 'R2_score': -0.5487079620361328,
 'explained_var_score': -0.08014082908630371,
 'MAPE': 0.38651517033576965}